In [1]:
%matplotlib qt

import numpy as np
import pyxem as pxm
import hyperspy.api as hs
import matplotlib.pyplot as plt
import math
import random

In [2]:
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from diffsims.generators.rotation_list_generators import get_beam_directions_grid

In [3]:
grid_cub = get_beam_directions_grid("cubic", 0.1, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])

# Parameters necessary for simulating a template library
diffraction_calibration = 0.01
half_shape = (256//2, 256//2)
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration
# importing the structures
structure_matrix = diffpy.structure.loadStructure("Au_mp-81_conventional_standard.cif")

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

library_phases = StructureLibrary(["phase"], [structure_matrix], [grid_cub])

diff_lib_for_data = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.08)

/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/diffsims/generators/sphere_mesh_generators.py:515: RuntimeWarning: invalid value encountered in true_divide
  phi2 = sign * np.nan_to_num(np.arccos(x_comp / norm_proj))
  5%|▍         | 52/1081 [00:00<00:01, 515.90it/s]

Number of patterns:  1081


In [3]:
def full_lib_thing(simulations,inplane):
    allpatterns = hs.signals.Signal2D(np.zeros((1,simulations.size,256,256)))
    allpatterns.set_signal_type(signal_type="electron_diffraction")
    pattern = hs.signals.Signal2D(np.zeros((256,256)))
    pattern.set_signal_type(signal_type="electron_diffraction")

    for i in range(5):
        pattern = pointplotter(simulations,inplane,i,simulations.size)
        allpatterns.inav[i,:] = pattern
        pattern = hs.signals.Signal2D(np.zeros((256,256)))
        
    allpatterns.set_diffraction_calibration(0.02)
    return allpatterns

In [4]:
def random_lib_thing(simulations):
    allpatterns = hs.signals.Signal2D(np.zeros((1,4000,256,256)))
    allpatterns.set_signal_type(signal_type="electron_diffraction")
    pattern = hs.signals.Signal2D(np.zeros((256,256)))
    pattern.set_signal_type(signal_type="electron_diffraction")

    for i in range(4000):
        patnum = random.randint(0,simulations.size)
        pattern = pointplotter(simulations,random.randint(0,360),patnum,5)
        allpatterns.inav[i,:] = pattern
        pattern = hs.signals.Signal2D(np.zeros((256,256)))
        
    allpatterns.set_diffraction_calibration(0.01)
    return allpatterns

In [5]:
def grab_random_pattern(simulations):
    inplane = random.randint(0,360)
    template = random.randint(0,simulations.size-1)
    pattern = pointplotter(simulations,inplane,template,5)
    return pattern, inplane, template
def build_random_tilts(diff_lib,n):
    orientations = diff_lib['phase']['orientations']
    simulations = diff_lib['phase']['simulations']
    pattern_set = hs.signals.Signal2D(np.zeros((1,n,256,256)))
    pattern_set.set_signal_type(signal_type="electron_diffraction")
    eulers = np.zeros((n,3))
    template_indexs = np.zeros((1,n))
    for i in range(n):
        pattern, inplane, template = grab_random_pattern(simulations)
        euler = orientations[template].copy()
        euler[0] = euler[0] + inplane
        pattern_set.inav[i,0] = pattern
        eulers[i,:] = euler
        template_indexs[0,i] = template
    return pattern_set,eulers,template_indexs

In [6]:
from pyxem.utils.plotting_utils import get_template_cartesian_coordinates 
from diffsims.pattern.detector_functions import add_shot_and_point_spread

def pointplotter(simulations,inplane,template,sigma):
    pattern = hs.signals.Signal2D(np.zeros((256,256)))
    pattern.set_signal_type(signal_type="electron_diffraction")
    x, y, intensities = get_template_cartesian_coordinates(simulations[template],in_plane_angle=inplane,window_size=(255,255),center=(256//2,256//2))
    for i in range(x.size):
        pattern.isig[x[i],y[i]]=intensities[i]
    pattern = add_shot_and_point_spread(pattern.T, sigma, shot_noise=False)
    pattern = hs.signals.Signal2D(pattern)
    return pattern


In [19]:
randompatterns, eulers, template_indexs = build_random_tilts(diff_lib_for_data,4000)

In [22]:
def plot_matchedandtrue_Oh(num):
    
    
    oritrue = Orientation.from_euler(
        np.radians(eulers[:,:,:]),
        symmetry=symmetry.Oh
    )
    # orimatched = Orientation.from_euler(
    # np.radians(result['orientation'][num,:,0,:]),
    # symmetry=symmetry.Oh
    # )
    ckey = IPFColorKeyTSL(oritrue.symmetry)
    rgbz = ckey.orientation2color(oritrue)
    cmap = np.linspace(0, 1, oritrue.size)
    # v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
    v = Vector3d((0, 0, 1))
    oritrue.scatter("ipf", direction=v, c=cmap)
    # orimatched.scatter("ipf", direction=v, c=cmap)

In [27]:
oritrue = Orientation.from_euler(
    np.radians(eulers[:,:]),
    symmetry=symmetry.Oh
)

In [25]:
# simulations = diff_lib_for_data["phase"]["simulations"]
# allpatterns = full_lib_thing(simulations,0)

In [34]:
randompatterns.plot(cmap = 'viridis')

In [14]:
allpatternsprec.plot(cmap = 'inferno', scalebar=False)

In [40]:
def gridresdifflib(resolution):
    grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
    print("Number of patterns: ", grid_cub.shape[0])

    library_phases = StructureLibrary(["phase"], [structure_matrix], [grid_cub])


    diff_lib = lib_gen.get_diffraction_library(library_phases,
                                               calibration=diffraction_calibration,
                                               reciprocal_radius=reciprocal_radius,
                                               half_shape=half_shape,
                                               with_direct_beam=False,
                                               max_excitation_error=0.08)
    return diff_lib

In [39]:
# diff_lib_01 = gridresdifflib(0.1)
diff_lib_01 = diff_lib_for_data

In [41]:
diff_lib_025 = gridresdifflib(0.25)
diff_lib_05 = gridresdifflib(0.5)
diff_lib_1 = gridresdifflib(1)
diff_lib_2 = gridresdifflib(2)

/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/diffsims/generators/sphere_mesh_generators.py:515: RuntimeWarning: invalid value encountered in true_divide
  phi2 = sign * np.nan_to_num(np.arccos(x_comp / norm_proj))
  0%|          | 40/16471 [00:00<00:41, 396.74it/s]

Number of patterns:  16471


  1%|          | 46/4186 [00:00<00:09, 455.08it/s]    

Number of patterns:  4186


  4%|▎         | 38/1081 [00:00<00:02, 377.26it/s]  

Number of patterns:  1081


 13%|█▎        | 40/300 [00:00<00:00, 394.42it/s]   

Number of patterns:  300


In [37]:
randompatterns = randompatterns*100

In [142]:
randompatterns.plot()

In [42]:
def template_log_func(x):
        return np.log10(x + 0.01)


frac_keep = 1
n_best = 30
n_keep = None

delta_r = 0.5
delta_theta = 0.5
max_r = None
intensity_transform_function = template_log_func
find_direct_beam = False
direct_beam_position = None
normalize_image = True
normalize_templates = True

In [141]:
randompatterns.change_dtype('bool')

In [43]:
result01, phasedict = iutls.index_dataset_with_template_rotation(randompatterns,
                                                    diff_lib_01,
                                                    phases = ["phase"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=template_log_func,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed | 35min 12.5s


In [50]:
result025, phasedict = iutls.index_dataset_with_template_rotation(randompatterns,
                                                    diff_lib_025,
                                                    phases = ["phase"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=template_log_func,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed |  6min 38.8s


In [51]:
result05, phasedict = iutls.index_dataset_with_template_rotation(randompatterns,
                                                    diff_lib_05,
                                                    phases = ["phase"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=template_log_func,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed |  2min 45.4s


In [52]:
result1, phasedict = iutls.index_dataset_with_template_rotation(randompatterns,
                                                    diff_lib_1,
                                                    phases = ["phase"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=template_log_func,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed |  3min  0.8s


In [137]:
result2, phasedict = iutls.index_dataset_with_template_rotation(randompatterns,
                                                    diff_lib_2,
                                                    phases = ["phase"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=template_binary_func,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

/home/joseph/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass threshold=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/joseph/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass threshold=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: `dtype` inference failed in `map_blocks`.

Please specify the dtype explicitly using the `dtype` kwarg.

Original error is below:
------------------------
ValueError('Found array with dim 4. Estimator expected <= 2.')

Traceback:
---------
  File "/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/dask/array/core.py", line 395, in apply_infer_dtype
    o = func(*args, **kwargs)
  File "<ipython-input-136-fb19f1b603f0>", line 3, in template_binary_func
    binarize(x,0, copy = False)
  File "/home/joseph/.local/lib/python3.9/site-packages/sklearn/utils/validation.py", line 74, in inner_f
    return f(**kwargs)
  File "/home/joseph/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py", line 2086, in binarize
    X = check_array(X, accept_sparse=['csr', 'csc'], copy=copy)
  File "/home/joseph/.local/lib/python3.9/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/home/joseph/.local/lib/python3.9

In [10]:
corrscores = result['correlation'][:,:,0][0]
indexes = np.linspace(0,simulations.size-1,num=simulations.size,dtype=int)

NameError: name 'result' is not defined

In [264]:
indexedarray =result2["template_index"][:,:,0]

onlytrueindexes = np.zeros(0,np.int16)
onlyfalseindexes = np.zeros(0,np.int16)
for i in range(indexes.size):
    if indexes[i] == indexedarray[0][i]:
        onlytrueindexes = np.append(onlytrueindexes, indexes[i])
    else:
        onlyfalseindexes = np.append(onlyfalseindexes, indexes[i])

In [266]:
onlytrueindexes.size

1

In [7]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d, AxAngle, Miller
from pyxem.signals.indexation_results import results_dict_to_crystal_map

In [8]:
from orix.crystal_map import Phase
from orix.plot import IPFColorKeyTSL
from orix.quaternion import Orientation, symmetry
from orix.vector import Miller, Vector3d

plt.rcParams.update({
    "figure.figsize": (15, 10),
    "axes.grid": False,
#     "font.size": 20,
    "lines.markersize": 15,
})

In [46]:
xmap01 = results_dict_to_crystal_map(result01,phasedict)

In [54]:
xmap025 = results_dict_to_crystal_map(result025,phasedict)
xmap05 = results_dict_to_crystal_map(result05,phasedict)
xmap1 = results_dict_to_crystal_map(result1,phasedict)
xmap2 = results_dict_to_crystal_map(result2,phasedict)

In [58]:
save(
    filename="simgridxmap2.h5",
    object2write=xmap2,
    overwrite=True,  # Default is False
)

In [48]:
import pickle
filename = "simgrideulers.pickle"
pickling_on = open("simgrideulers.pickle","wb")
pickle.dump(eulers, pickling_on)
pickling_on.close()

In [49]:
pickling_on = open("simgridtemplates.pickle","wb")
pickle.dump(template_indexs, pickling_on)
pickling_on.close()

In [89]:
xmap025.phases[0].space_group = 225

In [97]:
def something(xmap):
    oritrue = Orientation.from_euler(
        np.radians(eulers[:,:]),
        symmetry=symmetry.Oh
    )
    xmap.phases[0].space_group = 225
    orimatched = xmap.orientations
    misangles = np.degrees((oritrue- orimatched).angle.data)
    misangles = misangles.flatten()
    
    return(oritrue,orimatched,misangles)

In [69]:
oritrue = Orientation.from_euler(
    np.radians(eulers[:,:]),
    symmetry=symmetry.Oh
)
orismatched = xmap01.orientations

In [150]:
oritrue, orimatched, misangles = something(xmap01)

In [151]:
ckey = IPFColorKeyTSL(oritrue.symmetry)
rgbz = ckey.orientation2color(oritrue)
cmap = np.linspace(0, 1, oritrue.size)
# v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
v = Vector3d((0, 0, 1))
oritrue.scatter("ipf", direction=v, c=misangles, s= 50, vmax = 5)
orimatched.scatter("ipf", direction=v, c=misangles, s =50, vmax = 5)

In [123]:
misangles.mean()

2.6304873397753537

In [124]:
misangles.std()

9.350701061763043

In [125]:
np.count_nonzero(misangles < 10)/misangles.size

0.9655

In [126]:
misanglesnooutlier = np.delete(misangles, np.where(misangles > 10))

In [127]:
misanglesnooutlier.std()

0.4558925600244758

In [128]:
misanglesnooutlier.mean()

0.9092017621621805

In [88]:
stdmisangles01 = 0.36754406576541193
meanmisangles01 = 0.5303043832308559

In [113]:
stdmisangles025 = 0.3597067733998186
meanmisangles025 = 0.5225212797839106

In [129]:
stdmisangles05 = 0.36379143996822483
meanmisangles05 = 0.5406188207331206

In [133]:
stdmisangles1 = 0.3762584352555345
meanmisangles1 = 0.6361048409776127

In [132]:
stdmisangles2 = 0.4558925600244758
meanmisangles2 = 0.9092017621621805

In [9]:
# [0.36754406576541193,0.3597067733998186,0.36379143996822483,0.3762584352555345,0.4558925600244758]
means = [0.5303043832308559,0.5225212797839106,0.5406188207331206,0.6361048409776127,0.9092017621621805]

In [11]:
plt.rcParams.update({
    "figure.figsize": (10, 10),
    "lines.markersize": 10,
    "font.size": 20,
    "axes.grid": True,
})

In [10]:
plt.plot([0.1,0.2,0.5,1,2],means,'bo')

In [14]:
plt.plot([0.1,0.25,0.5,1,2],means,'bo',label = 'Au')

plt.gca().set(xlabel='Library Step Size($\degree$)', ylabel='Mean Misorientation($\degree$)')
plt.xticks(fontsize=12); plt.yticks(fontsize=12)
plt.legend()

In [29]:
orifirst = xmap.orientations
store = np.zeros((4186,30))
for i in range (30):
    corr_weight = xmap.correlation[:,i]/xmap.correlation[:,0]
    orinext = Orientation(xmap.rotations[:,i],symmetry=symmetry.Oh)
    store[:,i] = np.degrees((orinext - orifirst).angle.data) 
mean_misori_weighted = np.mean(store[:,1::], axis = 1)

In [26]:
mean_misori_thing = np.mean(store[:,:], axis = 1)

In [ ]:
xmap.correlation[100,:]

array([0.00902749, 0.00878098, 0.0085545 , 0.00853867, 0.00847657,
       0.00839564, 0.00838702, 0.00837368, 0.00831594, 0.00826802,
       0.00824405, 0.00824348, 0.00815314, 0.00813394, 0.00809084,
       0.00807126, 0.00800862, 0.00800147, 0.00799479, 0.0078176 ,
       0.00776885, 0.00754935, 0.00754445, 0.00749832, 0.00749243,
       0.00743447, 0.00742762, 0.00707373, 0.0070406 , 0.00699684])

In [43]:
realiabity = 100*(1-(result05['correlation'][:,:,1]/result05['correlation'][:,:,0]))

In [23]:
orifull = Orientation.from_euler(
    np.radians(diff_lib_for_data['phase']['orientations'][:,:]),
    symmetry=symmetry.Oh
)
orimatched = Orientation.from_euler(
    np.radians(result05['orientation'][0,:,0,:][:]),
    symmetry=symmetry.Oh
)

In [84]:
orifull5 = Orientation.from_euler(
    np.radians(diff_lib['phase']['orientations'][np.where(misangles > 5),:]),
    symmetry=symmetry.Oh
)
orimatched5 = Orientation.from_euler(
    np.radians(result['orientation'][0,:,0,:][np.where(misangles > 5)]),
    symmetry=symmetry.Oh
)

In [18]:
result['orientation'][0,:,0,:][np.where(misangles > 5)]

array([[180.        ,  17.        ,  90.        ],
       [180.        ,  17.        ,  90.        ],
       [177.        ,  17.        ,  90.        ],
       [174.5       ,  17.        ,  90.        ],
       [180.        ,  36.8217775 ,  64.27926652],
       [178.5       ,  36.8217775 ,  64.27926652],
       [179.5       ,  36.8217775 ,  64.27926652],
       [177.5       ,  36.8217775 ,  64.27926652],
       [178.5       ,  36.8217775 ,  64.27926652],
       [179.5       ,  36.8217775 ,  64.27926652],
       [176.5       ,  36.8217775 ,  64.27926652],
       [177.        ,  36.8217775 ,  64.27926652],
       [178.        ,  36.8217775 ,  64.27926652],
       [180.        ,  38.09330122,  45.        ],
       [181.        ,  38.09330122,  45.        ],
       [180.        ,  38.09330122,  45.        ],
       [184.5       , -32.81092169, -46.32470963],
       [  8.5       , -34.74134309, -50.0587642 ],
       [  9.        , -34.74134309, -50.0587642 ],
       [ 10.        , -34.74134

In [175]:
ckey = IPFColorKeyTSL(orifull.symmetry)
rgbz = ckey.orientation2color(orifull)
# cmap = np.linspace(0, 1, orimatched.size)
cmap = corrscores

In [250]:
v = Vector3d(((0, 0, 1)))
fig = orifull.scatter("ipf", direction=v,return_figure= True, s = 100, c = 'r')
orimatched.scatter("ipf", direction=v, figure = fig,s = 100, c = 'g', marker = 'o')

In [48]:
v = Vector3d(((0, 0, 1)))
fig2 = orifull.scatter("ipf", direction=v,return_figure= True, s = 50, c = misangles, cmap = 'viridis', vmax=5)
# orifull[4140].scatter("ipf", direction=v, figure = fig2,s = 100, c = 'r', marker = 'o')

In [30]:
v = Vector3d(((0, 0, 1)))
fig2 = orifull.scatter("ipf", direction=v,return_figure= True, s = 50, c = mean_misori_weighted, cmap = 'viridis', vmax=5)
# orifull[4140].scatter("ipf", direction=v, figure = fig2,s = 100, c = 'r', marker = 'o')

In [45]:
v = Vector3d(((0, 0, 1)))
fig2 = orifull.scatter("ipf", direction=v,return_figure= True, s = 50, c = realiabity, cmap = 'viridis')

In [147]:
misangles[2796]

39.26841871101089

In [173]:
fig.savefig('Matching001log.png',bbox_inches = 'tight', pad_inches = 0)

In [258]:
misangles2 = np.delete(misangles, np.where(misangles > 10))

In [259]:
print(np.std(misangles2))
print(np.mean(misangles2))
print(((misangles2)**2).mean())

0.49935468023968466
0.9120419218532182
1.0811755638949894
